#Installing all required Libraries

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00


In [ ]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install wrapt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importing the required Libraries

In [ ]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
from torch.optim import AdamW
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Importing Roberta-Base-Squad2 Model For Fine Tunning

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model = AutoModelForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')
tokenizer = AutoTokenizer.from_pretrained('deepset/roberta-base-squad2')

In [ ]:
model.to(device)
model.train()

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

# Importing the squad Dataset

In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset("squad", "plain_text", split="train")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


# Transforming the dataset in proper format


In [ ]:
train_features = []

for example in dataset:
    start_positions = []
    end_positions = []

    context = example["context"]
    questions = example["question"]
    answers = example["answers"]
    answer_start=0
    answer_end=0
    tokenized_context = tokenizer.tokenize(context)
    tokenized_question = tokenizer.tokenize(questions)
    tokenized_input = ["[CLS]"] + tokenized_question + ["[SEP]"] + tokenized_context + ["[SEP]"]
    inputs = tokenizer.convert_tokens_to_ids(tokenized_input)
    char_to_token = []
    token_index = 1  # Skip the [CLS] token
    for char_index, char in enumerate(context):
            if char != " " and char != "\n":
                char_to_token.append(token_index)
            if token_index < len(tokenized_input) - 1 and char_index + 1 < len(tokenized_input[token_index]):
                token_index += 1

    for answer in answers["answer_start"]:
        answer_start = answer
        token_start = char_to_token[min(answer_start, len(char_to_token) - 1)]
        token_start = min(token_start, len(inputs) - 1)
        start_positions.append(token_start)



    for text in answers["text"]:
        answer_end = answer_start + len(text) - 1
        token_end = char_to_token[min(answer_end, len(char_to_token) - 1)]
        token_end = min(token_end, len(inputs) - 1)
        end_positions.append(token_end)




Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
train_features.append(
            {
                "input_ids": inputs,
                "attention_mask": [1] * len(inputs),
                "start_positions": start_positions,
                "end_positions": end_positions,
            }
        )

# Choosing the best hyperparameters

In [ ]:
import torch
from transformers import  AdamW
from torch.utils.data import DataLoader, RandomSampler
from transformers import AdamW
import torch.optim as optim

# Define the batch size and number of training epochs
batch_size = 4
num_epochs = 50

# Convert the train_features list to a PyTorch DataLoader
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor([f["input_ids"] for f in train_features], dtype=torch.long),
    torch.tensor([f["attention_mask"] for f in train_features], dtype=torch.long),
    torch.tensor([f["start_positions"] for f in train_features], dtype=torch.long),
    torch.tensor([f["end_positions"] for f in train_features], dtype=torch.long),
)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Load the pre-trained BERT model for question answering
# model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')

# Set the model to training mode
model.train()

# Define the optimizer and learning rate
# optimizer = AdamW(model.parameters(), lr=3e-6)
optimizer = optim.Adamax(model.parameters(), lr=9e-6)


# Training the Model on the dataset

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, start_positions, end_positions = batch

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


# Deploying the Model in the Hugging Face Organization



In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForQuestionAnswering
from huggingface_hub import HfApi, HfFolder

In [ ]:
# another way to upload the files on hugging facce

output_dir = './fine_tuned_model_roberta'

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
tokenizer.save_pretrained(output_dir)
config = AutoConfig.from_pretrained("deepset/roberta-base-squad2")
config.save_pretrained(output_dir)
model.save_pretrained(output_dir)

In [ ]:
api = HfApi()
user = "your-huggingface-usernmae"

In [ ]:
repo_id = user + "/" + "roberta_fine_tuned_large"

In [ ]:
repo_name = "roberta-fine-tuned-large"  # Replace with your desired model name
api.create_repo(token="<your-token>", repo_id=repo_id)

RepoUrl('https://huggingface.co/techysanoj/roberta_fine_tuned_large', endpoint='https://huggingface.co', repo_type='model', repo_id='techysanoj/roberta_fine_tuned_large')

# Getting predictions from The Fine Tuned Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering


# Load the fine-tuned BERT model

model = AutoModelForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')
tokenizer = AutoTokenizer.from_pretrained('deepset/roberta-base-squad2')

# Prepare the input
question = "what are dogs?"
context = """dogs are animals"""
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Process the logits to obtain the predicted answer span
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

# Decode the predicted answer span
all_tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])

# Print the predicted answer
print("Question:", question)
print("Predicted Answer:", answer)


Question: what are dogs?
Predicted Answer:  animals


# Function to Tokenize the questions and get proper outputs

In [ ]:

def robert_question_answer(tokenizer, model,question, passage, max_len=500):
    inputs = tokenizer.encode_plus(question, passage, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index + 1]))
    return answer

def split_passage_and_process(tokenizer, model,question, passage, max_len=10000, chunk_size=500):
    # Split the passage into smaller chunks
    passage_chunks = [passage[i:i+chunk_size] for i in range(0, len(passage), chunk_size)]
    answers = []
    # Process each chunk separately
    for chunk in passage_chunks:
        answer = robert_question_answer(tokenizer, model,question, chunk, max_len=max_len)
        if answer != "Sorry!, I could not find an answer in the passage.":
            answers.append(answer)

    # Combine the answers from all chunks
    combined_answer = " ".join(answers)
    return combined_answer

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch

# Load the fine-tuned BERT model

model = AutoModelForQuestionAnswering.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained('deepset/roberta-base-squad2')

# Load the SQuAD validation dataset
datasetv = load_dataset("squad", "plain_text", split="validation[:500]")


Found cached dataset squad (/Users/shashwatbindal/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


# Predicting the outputs

In [ ]:

# # Load the fine-tuned BERT model

model = AutoModelForQuestionAnswering.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained('deepset/roberta-base-squad2')
# # Prepare the input


def generate_predictions(dataset, tokenizer, model):
    predictions = []
    for example in dataset:
        inputs = tokenizer.encode_plus(example["question"], example["context"], add_special_tokens=True, return_tensors="pt")
        input_ids = inputs["input_ids"].to(model.device)
        attention_mask = inputs["attention_mask"].to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

        start_index = torch.argmax(start_logits)
        end_index = torch.argmax(end_logits)

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index + 1]))
        predictions.append(answer.strip())

    return predictions


# Getting predictions

In [ ]:
predictions = generate_predictions(datasetv, tokenizer, model)


# Getting true answers from the squad dataset

In [ ]:
true_answers=[]
for ans in datasetv:
    true_answers.append(ans["answers"]["text"])
    # print(ans["answers"]["text"])


In [ ]:
predictions[5:10],true_answers[5:10]

(['golden anniversary',
  'February 7, 2016',
  'American Football Conference',
  'golden anniversary',
  'American Football Conference'],
 [['"golden anniversary"', 'gold-themed', '"golden anniversary'],
  ['February 7, 2016', 'February 7', 'February 7, 2016'],
  ['American Football Conference',
   'American Football Conference',
   'American Football Conference'],
  ['"golden anniversary"', 'gold-themed', 'gold'],
  ['American Football Conference',
   'American Football Conference',
   'American Football Conference']])

# Code for audio input of question from your microphone and getting answer in audio as well as in text form

In [ ]:
import sounddevice as sd
import soundfile as sf

# Set the audio settings
sample_rate = 44100
duration = 6 # Duration in seconds
output_file = "audio.wav"


In [ ]:

import speech_recognition as sr


In [ ]:
def askquestion(audio_file,output_file):
    print("Recording audio...")
    audio = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=1)
    sd.wait()

    # Save the audio to a file
    sf.write(output_file, audio, sample_rate)

    print(f"Audio saved to {output_file}")
    r = sr.Recognizer()

    # Path to the audio file

    # Load the audio file
    with sr.AudioFile(audio_file) as source:
        # Read the audio data
        audio = r.record(source)
    text=""

    try:
        # Recognize speech from the audio
        text = r.recognize_google(audio)
        print("Transcription:", text)
    except sr.UnknownValueError:
        print("Speech recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

    return text


In [ ]:
from gtts import gTTS

def text_to_speech(text, output_file):
    # Create a gTTS object with the text and desired language
    tts = gTTS(text=text, lang='en')

    # Save the audio to a file
    tts.save(output_file)

# Example usage

output_file = "output.mp3"

text_to_speech(text, output_file)

In [ ]:
import pygame

def play_mp3(file_path):
    pygame.mixer.init()
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()

In [ ]:
def Passageready():

    output_file2="passage.mp3"
    text_to_speech("Give the passage you want to ask questions from", output_file2)
    play_mp3(output_file2)
    passage=input("Enter the Passage: ")
    return passage


In [ ]:
def QuestionAnswer(passage):

    output_file="Ask.mp3"
    text_to_speech("Ask the question", output_file)
    play_mp3(output_file)
    question=askquestion("question.wav","question.wav")
    ans  = split_passage_and_process( tokenizer, model,question, passage)
    output_file3="answer.mp3"
    text_to_speech("The Answer to the question....: "+question+"..."+"is....:"+ans+"...", output_file3)
    play_mp3(output_file3)
    return ans


In [ ]:
# passage=Passageready()

In [ ]:
# ans=QuestionAnswer(passage)

In [ ]:
# pip install opencv-python


In [ ]:
# pip install --upgrade pytesseract

In [ ]:
import cv2
import pytesseract

# Load the image using OpenCV
image = cv2.imread('/Users/shashwatbindal/Downloads/WhatsApp Image 2023-06-15 at 13.42.55.jpeg')

# Preprocess the image (if required)
# Apply any necessary preprocessing techniques such as resizing, noise removal, etc.

# Perform OCR using Tesseract
text = pytesseract.image_to_string(image)

# Save the extracted text to a text file
with open('output.txt', 'w') as file:
    file.write(text)

print('Text extracted and saved to output.txt')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Text extracted and saved to output.txt


In [ ]:
true_answers[:5],predictions[:5]


([['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
  ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers'],
  ['Santa Clara, California',
   "Levi's Stadium",
   "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."],
  ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
  ['gold', 'gold', 'gold']],
 ['Denver Broncos',
  'Carolina Panthers',
  "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California",
  'Denver Broncos',
  'gold'])

# Exact Match Score of the fine tuned model (5-7% increase from base model)

In [ ]:
exact_match_count = 0
total_examples = len(true_answers)

for true_answer, prediction in zip(true_answers, predictions):
    if prediction in true_answer:
        exact_match_count += 1

exact_match_score = exact_match_count / total_examples

# Print the exact match score
print("Exact match score:", exact_match_score)

Exact match score: 0.89
